# CHSH Game: Testing the Limits of Reality with Quantum Mechanics 
Would you like to re-create some of the experiments that led to the 2022 Nobel Prize in Physics? And how about proving Einstein wrong while we’re at it? If that sound fun, keep reading - this notebook explores the CHSH game, a thought experiment and quantum circuit that demonstrates how quantum entanglement can lead to results that defy classical intuition.

## Background
### Einstein Bohr debates
Quantum particles exhibit some very weird (to us humans) phenomenon. Chief among the freaky properties of quantum particles are superposition and entanglement. These properties are the key reason why Quantum Computing works and is better at certain tasks than classical computing. However, quantum theory when first proposed did not find support from some key scientists – Albert Einstein the most prominent one among them. Einstein strongly believed that that quantum mechanics was an incomplete theory. What appeared to us as particles behaving probabilistically was in fact just proof that the theory was incomplete and that other hidden variables were at play which we were ignorant about.

In 1935, Einstein, Podolsky, and Rosen (EPR) published a thought-provoking paper questioning the completeness of quantum mechanics. The EPR paradox was rooted in the apparent contradiction between quantum mechanics' probabilistic predictions and the principle of locality — the idea that objects are only influenced by their immediate surroundings. EPR introduced the concept of "hidden variables," hypothesizing that quantum mechanics might be an incomplete description of reality.

<img src="./Images/EPR.jpg" alt="Einstein disagrees with Quantum theory, considers it incomplete"/>

This is what Einstein referred to when he famously said, “God does not play dice”. In response, Niels Bohr supposedly asked him to stop telling God what to do with his dice. Bohr defended the Copenhagen interpretation, arguing that quantum mechanics provides a complete description of nature but requires abandoning classical notions of determinism and locality. This philosophical clash between Bohr and Einstein set the stage for one of the most profound debates in physics.

<img src="./Images/Einstein_Bohr.jpg" alt="Einstein and Bohr, deep in discussion in December 1925" width="900"/>


### Bell's Inequality: A Mathematical Revolution
While the Einstein-Bohr debate raged, not many had ideas to experimentally prove either of them right. How do you experimentally prove a phenomenon that collapses when observed? Until John Bell came along. John Bell revolutionized the debate in 1964 by deriving his famous inequality. Bell's theorem mathematically formalized the distinction between predictions of quantum mechanics and those of any local hidden variable theory. 

<img src="./Images/John_Bell.png" alt="John Bell, with his famous inequality in on the blackboard" width="900"/>

### Clauser's Experiments and the Nobel Prize
In the 1970s, John Clauser, building on Bell’s work, designed experiments to test these predictions using the CHSH inequality. These experiments, conducted with unprecedented precision, consistently violated the inequality, affirming quantum mechanics' predictions and discrediting local hidden variable theories. Clauser's contributions, alongside those of Alain Aspect and Anton Zeilinger, earned him the 2022 Nobel Prize in Physics, cementing the CHSH inequality's place in the annals of science.

<img src="./Images/Clauser.jpg" alt="John Clauser, who won the 2022 Physics Nobel prize" width="900"/>

### The Mathematical Framework of the CHSH Inequality
The CHSH game builds on Bell's theorem, offering a concrete formulation for experimental tests. It introduces the concept of a two-player game to explore quantum mechanics' predictions versus classical expectations. The CHSH game is a thought experiment involving two players, Alice and Bob, who are given bits x and y (0 or 1) as inputs respectively and must return bits a and b as outputs. The players win if their outputs satisfy the condition:

<img src="./Images/Equation1.png" alt="Equation1"/>

where the plus symbol inside the circle denotes the XOR operation and the dot symbol denotes the AND operation.

In a classical setting, Alice and Bob can only use shared randomness (local hidden variables) to correlate their outputs. The maximum probability of winning the CHSH game under classical mechanics is 75%. This limitation can be expressed as the CHSH inequality:

<img src="./Images/Equation2.png" alt="Equation2"/>

where E(Ax, By) denotes the correlation between Alice's and Bob's outputs for inputs x and y.

Quantum mechanics allows Alice and Bob to share an entangled state, enabling them to achieve a winning probability of approximately 85.4%. The corresponding quantum expression for the CHSH inequality is:

<img src="./Images/Equation3.png" alt="Equation3"/>

Thus, experimental observation can show us whether we achieve a probability of 75% or a higher 85% - if we can empirically show that our probability for such an experiment is higher than 75%, that disproves the "hidden local variable" hypothesis and proves that quantum mechanics is as weird and fun as we believe it to be.

## Implementing the CHSH game in Qiskit
Clauser et al did not have access to quantum computers and had to do these experiments the difficult way - by actually entangling photons and polarization measurements. However, today we have access to quantum computers thanks to IBM and can easily conduct these experiments using Qiskit. Here is an implementation of the CHSH inequality in Qiskit.

First, let us import IBM Qiskit. Qiskit is an open-source, Python-based high performance software stack for quantum computing. The Qiskit SDK allows us to run quantum programs on IBM's quantum hardware or classical simulators.

Run the below cell to ensure you have Qiskit installed. You should see a version of greater than 2.0.0 when you run this cell.

In [15]:
# Checking if Qiskit is correctly installed
import qiskit
qiskit.__version__

'2.0.1'

Great job! Now let us import the rest of the libraries we need for this experiment.

In [22]:
# Importing the libraries we need for this experiment
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from numpy import pi
from numpy.random import randint

We will now run the experiment on the AerSimulator provided by Qiskit. For small enough number of qubits, simulators use our computer's CPU to simulate the quantum nature of qubits. This is what we will do first.

In [17]:
# For running the job on a quantum simulator
backend = AerSimulator()

### Define the game
Lets now define the game that we are playing, essentially simulating our thought experiment. The CHSH game is a two-player cooperative game played by Alice and Bob against a referee. Each round works like this:

- The referee sends two random bits: x to Alice and y to Bob (each ∈ {0, 1}).
- Alice and Bob respond with their own bits a and b, without communicating.
- They win the game if their answers satisfy:
𝑎 ⊕ 𝑏 = 𝑥 ⋅ 𝑦

In other words, for Alice and Bob to win the game:
- If both x and y are 1, then a and b must be different
- Otherwise, a and b must be the same

This function simulates one round of the game: The referee gives two random bits x and y (chosen using randint). The strategy function returns Alice and Bob's responses a and b depending on whether we are playing the classical version or quantum version. The referee checks if they win using the CHSH win condition: a ^ b == x & y.

In [10]:
def chsh_game(strategy):
 
    # Referee chooses x and y randomly
    x, y = randint(0, 2), randint(0, 2)

    # Use strategy to choose a and b
    a, b = strategy(x, y)

    # Referee decides if Alice and Bob win or lose
    if (a ^ b) == (x & y):
        return 1  # Win
    else:
        return 0  # Lose

### Define the classical strategy
Classically, the best option for Alice and Bob is to return the same bit always irrespective of the values of x or y. Remember, Alice and Bob cannot communicate with each other. Since they cannot know what the other received from the referee, they decide on the best strategy before starting the game. Their best strategy is to always respond with the same bit. Since responding with the same bit gives them a win in 3 out of 4 situations:
- When x = 0, y = 0, Alice and Bob need to return the same a and b ==> Win for the classical strategy
- When x = 0, y = 1, Alice and Bob need to return the same a and b ==> Win for the classical strategy
- When x = 1, y = 0, Alice and Bob need to return the same a and b ==> Win for the classical strategy
- When x = 1, y = 1, Alice and Bob need to return different a and b ==> Loss for the classical strategy

Thus, with the classical strategy, Alice and Bob can win at most 75% of the time assuming x and y are chosen randomly and they play sufficient number of games.

In [11]:
def classical_strategy(x, y):
    return 0, 0 #Always returns the same a and b irrespective of x and y

### Define the quantum strategy
However, using quantum entanglement, Alice and Bob can achieve a higher win rate. They do this by sharing a pair of entangled qubits before the game begins. This entanglement allows their measurement results to be correlated in ways that classical physics can’t explain.

Alice and Bob start with a Bell state - an entangled pair created using a Hadamard gate (H) and a CNOT gate:

<img src="./Images/Bell_State.png" alt="Bell State"/>

When Alice receives her input bit x, she chooses her measurement angle:
- If x = 0, she measures in the 0° basis (ry(0))
- If x = 1, she measures in the 90° basis (ry(π/2))

Bob, based on his input bit y, measures in a rotated basis:
- If y = 0, he uses an angle of 45° (ry(π/4))
- If y = 1, he uses −45° (ry(−π/4))

Their outputs a and b are determined from the quantum measurements. Lets write the quantum_strategy function that does this and returns Alice's and Bob's responses based on the quantum measurements of the entangled Bell pair. 

In [12]:
def quantum_strategy(x, y):
    
    qc = QuantumCircuit(2,  2)
    qc.h(0)
    qc.cx(0, 1)
    
    # Alice
    if x == 0:
        qc.ry(0, 0)
    else:
        qc.ry(pi / 2, 0)
    qc.measure(0, 0)

    # Bob
    if y == 0:
        qc.ry(pi / 4, 1)
    else:
        qc.ry(-pi / 4, 1)
    qc.measure(1, 1)
    cc = transpile(qc, backend)
    job = backend.run(cc, shots=1)
    result = list(job.result().get_counts(cc))

    # Return a and b based on result:
    if result[0] == "00":
        return 0, 0
    elif result[0] == "01":
        return 0, 1
    elif result[0] == "10":
        return 1, 0
    else:
        return 1, 1

### The result
Now let us run this game 1000 times (shown by the variable NUM_GAMES below - you can change this number to see how playing more rounds or less rounds may impact our results)

In [14]:
NUM_GAMES = 1000
TOTAL_SCORE_CLASSICAL = 0
TOTAL_SCORE_QUANTUM = 0

for i in range(NUM_GAMES):
    TOTAL_SCORE_CLASSICAL += chsh_game(classical_strategy)
    TOTAL_SCORE_QUANTUM += chsh_game(quantum_strategy)

print("Classical Strategy - Fraction of games won:", TOTAL_SCORE_CLASSICAL / NUM_GAMES)
print("Quantum Strategy   - Fraction of games won:", TOTAL_SCORE_QUANTUM / NUM_GAMES)

Classical Strategy - Fraction of games won: 0.764
Quantum Strategy   - Fraction of games won: 0.837


As we had hypothesized - the classical strategy should show that Alice and Bob win the game about 75% of the time. If your classical win rate is slightly higher or lower - this is due to randomness.For example, even though a fair coin has a 50% chance of landing heads, tossing it just 10 times may yield 7 heads or only 3. This is why we simulate the game over 1,000 trials — the more repetitions, the closer the empirical win rate approaches the theoretical value.

Now, when we run the quantum strategy, something remarkable happens: the win rate jumps to approximately 85%. This isn't a fluke — it reflects a deep feature of quantum mechanics. Alice and Bob share a pair of entangled qubits, and by carefully choosing their measurement angles based on their respective input bits, they exploit quantum correlations that have no classical explanation. Their measurement outcomes are not just random — they’re quantum-coordinated in a way that increases their chances of meeting the CHSH win condition. This optimal quantum strategy reaches the so-called Tsirelson bound, which sets the maximum possible win rate in the CHSH game allowed by quantum physics:

<img src="./Images/Quantum_Win_Rate.png" alt="Max win rate"/>

This bound is not just higher than the classical limit — it's fundamentally unreachable by any classical means. No hidden-variable theory or pre-agreed strategy can surpass the 75% threshold. Only entanglement enables such performance. 

Until now, we’ve used the Aer simulator to model a quantum computer, with all computations running locally on our own CPU. But thanks to modern cloud-based platforms, we can now go a step further. From the comfort of our homes, we have access to real quantum hardware provided by IBM. Let’s now run the same CHSH experiment on an actual IBM quantum computer and see how it performs in the real world.

Use the instructions provided in the "Installation.ipynb" file to get your own API token and plug it in the below cell where it says 'YOUR_API_TOKEN' (plug in your API token within the single quotes).

In [ ]:
# For running the job on the IBM's Quantum Computer
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    token='YOUR_API_TOKEN'  
)
backend = service.least_busy(simulator=False, operational=True)

Now we run the quantum_strategy function again to ensure we are using the measurements from real quantum hardware instead of our local simulator.

In [ ]:
def quantum_strategy(x, y):
    
    qc = QuantumCircuit(2,  2)
    qc.h(0)
    qc.cx(0, 1)
    
    # Alice
    if x == 0:
        qc.ry(0, 0)
    else:
        qc.ry(pi / 2, 0)
    qc.measure(0, 0)

    # Bob
    if y == 0:
        qc.ry(pi / 4, 1)
    else:
        qc.ry(-pi / 4, 1)
    qc.measure(1, 1)
    cc = transpile(qc, backend)
    job = backend.run(cc, shots=1)
    result = list(job.result().get_counts(cc))

    # Return a and b based on result:
    if result[0] == "00":
        return 0, 0
    elif result[0] == "01":
        return 0, 1
    elif result[0] == "10":
        return 1, 0
    else:
        return 1, 1

And now, lets run the games again. We are using a much smaller number of NUM_GAMES here, because we typically have only 10 mins of QPU time for free from IBM. You can increase or decrease the NUM_GAMES based on how much QPU times you have remaining and see the results.

In [ ]:
NUM_GAMES = 10
TOTAL_SCORE_CLASSICAL = 0
TOTAL_SCORE_QUANTUM = 0

for i in range(NUM_GAMES):
    TOTAL_SCORE_CLASSICAL += chsh_game(classical_strategy)
    TOTAL_SCORE_QUANTUM += chsh_game(quantum_strategy)

print("Classical Strategy - Fraction of games won:", TOTAL_SCORE_CLASSICAL / NUM_GAMES)
print("Quantum Strategy   - Fraction of games won:", TOTAL_SCORE_QUANTUM / NUM_GAMES)


## Conclusion
The CHSH game isn't just a theoretical exercise — it's a real, testable way to distinguish between classical and quantum worlds. The fact that our quantum strategy reliably beats the classical 75% limit offers experimental validation of quantum mechanics. This disproves Einstein’s local hidden variable theories and is one of the clearest demonstrations that the universe behaves in a genuinely non-classical, nonlocal way. And now, you have also re-created the experiments which proved Einstein wrong and led to a Nobel prize, all from the comfort of your own home!



## References
- J. S. Bell (1964). On the Einstein Podolsky Rosen Paradox. Physics Physique, 1(3), 195–200. https://cds.cern.ch/record/111654
- J. F. Clauser, M. A. Horne, A. Shimony, R. A. Holt (1969). Proposed Experiment to Test Local Hidden-Variable Theories. Physical Review Letters, 23(15), 880. https://doi.org/10.1103/PhysRevLett.23.880
- The Nobel Prize in Physics 2022 – Press Release. NobelPrize.org. https://www.nobelprize.org/prizes/physics/2022/press-release/
- Michael Nielsen & Isaac Chuang (2010). Quantum Computation and Quantum Information. Cambridge University Press.
(Chapters on Bell inequalities and entanglement)
- Qiskit Documentation – Official docs for Qiskit, Aer, and IBM Runtime. https://docs.quantum.ibm.com/
- Scott Aaronson’s Lecture Notes – Lecture 14: Nonlocal Games. https://www.scottaaronson.com/qclec.pdf

Qiskit v.2.0.1 was used to create this tutorial.

